# Processing Notebook

The purpose of this notebook is to process and aggregate the *.csv* files provided by `twitch_stream_bot.py`.


In [1]:
import pandas as pd
import os

processing_old_twitch_API = False

All the *.csv* files are located in the `sync` folder.

In [2]:
if processing_old_twitch_API:
    csv_path = './sync/tagIds/' # For old Twitch API
else:
    csv_path = './sync/tagIds_tags/' # For new Twitch API

csv_dfs = [pd.read_csv(csv_path + file) for file in os.listdir(path=csv_path) if file.endswith('csv')]

df = pd.concat(csv_dfs, ignore_index=True)

print(f'There are {len(df) - len(df.id.drop_duplicates())} duplicates')

print('Deleting duplicates...')
df.sort_values('_custom_ended_at', inplace=True)
df = df.groupby(['id']).apply(lambda df_ : df_.iloc[-1]).reset_index(drop=True)

print(f'There are {len(df) - len(df.id.drop_duplicates())} duplicates')

df.sort_values('started_at', inplace=True, ignore_index=True)


There are 29 duplicates
Deleting duplicates...
There are 0 duplicates


Printing some infos about the data.

In [3]:
print(f'''{len(df)} streams
{len(df.groupby('user_id'))} different broadcasters
first started live : {df.iloc[0]['started_at']}
last started live : {df.iloc[-1]['started_at']}
''')

1725 streams
711 different broadcasters
first started live : 2023-01-05T13:18:16Z
last started live : nan



Saving the dataframe in the `../Data` folder for further use.

In [4]:
if processing_old_twitch_API:
    df.to_csv('../Data/twitch_streams_vtuber_tagIds.csv', index=False)
else:
    df.to_csv('../Data/twitch_streams_vtuber_tagIds_tags.csv', index=False)
